In [1]:
!pip install pandas
!pip install sodapy

In [2]:
import pandas as pd
from sodapy import Socrata

In [71]:
# api endpoint
api_url = "https://data.colorado.gov/resource/4ykn-tg5h.json"
app_token = "0NT6cCdGg6VVAj2pRLjmAqDFu"
username = "glissmann.mark@gmail.com"
password = "K2PRH@mUDwgbfsV"
entityid = "18881009142"
api_key = "gxc0jsptwdopcqxffpl21c8b"
api_secret_key = "2w9yp2sqyp3d7nxkidj8v65e6ow7mos3e7bnuc64rqaiccyc4c" 


In [67]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.colorado.gov", None)


# authenticated client (needed for non-public datasets):
client = Socrata("data.colorado.gov",
                 app_token,
                 username=username,
                 password=password)


In [68]:
# First 2000 results, returned as CSV from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("4ykn-tg5h")

In [69]:

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [70]:
len(results_df)

1000

In [72]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd


# Set the parameters for pagination
limit = 1000  # Number of records per request
offset = 0    # Starting offset
all_data = [] # List to store all records

while True:
    # Define the parameters with the current offset
    params = {
        '$limit': limit,
        '$offset': offset
    }

    # Make the API request with HTTP Basic Authentication
    response = requests.get(api_url, params=params, auth=HTTPBasicAuth(username, password))
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()  # Parse the JSON response
        if not data:
            break  # Exit loop if no more data is returned
        all_data.extend(data)  # Add the retrieved data to the list
        offset += limit        # Increment the offset for the next request
    else:
        print(f'Error: {response.status_code}, {response.text}')
        break


KeyboardInterrupt: 

In [73]:

# Convert the list of all records to a pandas DataFrame
df = pd.DataFrame(all_data)
print(df.head())
print(f'Total records retrieved: {len(df)}')


      entityid                                         entityname  \
0  18861217679  DENVER UNION CORPROATION, Dissolved January 17...   
1  18881009142        The Colorado State Bee-Keepers' Association   
2  18911012261  GRADEN MERCANTILE COMPANY (THE), Dissolved May...   
3  18911112108                              Stearns-Roger Limited   
4  18961000001                             PARK RESERVOIR COMPANY   

            principaladdress1     principalcity principalstate  \
0    1512 LARIMER STREET #760            Denver             CO   
1       9250 Rector Leader Rd             Byers             CO   
2               777 Main Ave.           Durango             CO   
3  4520 Centennial Blvd #1105  Colorado Springs             CO   
4               980 W Main St         Cedaredge             CO   

  principalzipcode                entitystatus jurisdictonofformation  \
0            80202       Voluntarily Dissolved                     CO   
1            80103               Good Stan